In [1]:
# ============================================
# Osmoz 1.22 — Import Amazon (.xlsx/.csv/.tsv/.txt) + Étiquettes PDF (A4, 8/page)
# Modifié : lien de téléchargement visible, PDF : adresse + grande, wrapping 20 chars,
# logo collé à gauche et 20% plus grand, logo par défaut depuis URL si pas d'upload.
# ============================================

import sys, subprocess, importlib, io, os, re, math, traceback, csv, urllib.request
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass

# ---------- Install deps if missing ----------
def _pip_install(pkg):
    import importlib.util
    if importlib.util.find_spec(pkg) is None:
        print(f"Installation de {pkg}…")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

for p in ["pandas", "ipywidgets", "openpyxl", "chardet", "reportlab", "pillow"]:
    _pip_install(p)

import pandas as pd, chardet
from ipywidgets import (
    VBox, HBox, HTML, Button, FileUpload, Output, Layout, Checkbox
)
from IPython.display import display, FileLink, clear_output
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import mm
from reportlab.lib.utils import ImageReader
from PIL import Image

# ========== UI container (allow rebuild on RESET) ==========
app_box = VBox()

# ---------- Journal ----------
log = Output()
def log_print(*args, **kw):
    with log:
        print(*args, **kw)

# ---------- Utils bytes / text ----------
def to_bytes(obj) -> bytes:
    if obj is None:
        return b""
    if isinstance(obj, (bytes, bytearray)):
        return bytes(obj)
    try:
        if isinstance(obj, memoryview):
            return obj.tobytes()
    except NameError:
        pass
    try:
        return bytes(obj)
    except Exception:
        return b""

def smart_decode(b: bytes) -> Tuple[str, str]:
    if not b:
        return "", "utf-8"
    try:
        return b.decode("utf-8"), "utf-8"
    except Exception:
        enc = chardet.detect(b).get("encoding") or "utf-8"
        try:
            return b.decode(enc, errors="replace"), enc
        except Exception:
            return b.decode("latin-1", errors="replace"), "latin-1"

# ---------- Parsing helpers ----------
AMZ_COLS_BASE = [
    "order-id","order-item-id","purchase-date","payments-date","reporting-date","promise-date","days-past-promise",
    "buyer-email","buyer-name","payment-method-details","buyer-phone-number","sku","number-of-items","product-name",
    "quantity-purchased","quantity-shipped","quantity-to-ship","ship-service-level","recipient-name",
    "ship-address-1","ship-address-2","ship-address-3","ship-city","ship-state","ship-postal-code","ship-country",
]

def normalize_text_table(txt: str) -> str:
    txt = txt.replace("\r\n", "\n").replace("\r", "\n")
    txt = re.sub(r'(?<!\n)(\d{3}-\d{7}-\d{7}\t)', r'\n\1', txt)
    txt = txt.replace("\x00", "")
    return txt

def read_table_flex(file_bytes: bytes, filename: str):
    meta = {}
    name = filename.lower()
    if name.endswith(".xlsx"):
        df = pd.read_excel(io.BytesIO(file_bytes), engine="openpyxl")
        meta['parser'] = 'xlsx'
        return df, meta
    text, enc = smart_decode(file_bytes)
    meta['encoding'] = enc
    text = normalize_text_table(text)
    if name.endswith(".tsv") or ("\t" in text and text.count("\t") > text.count(",")):
        delim = "\t"
    else:
        delim = ","
    lines = text.splitlines()
    if not lines:
        return pd.DataFrame(), meta
    reader = csv.reader(lines, delimiter=delim)
    rows = list(reader)
    if len(rows) >= 1:
        cols = rows[0]
        data = rows[1:]
    else:
        cols = [f"col{i}" for i in range(len(rows[0]))]
        data = rows
    norm = []
    for r in data:
        if len(r) > len(cols):
            r = r[:len(cols)]
        elif len(r) < len(cols):
            r = r + [""]*(len(cols)-len(r))
        norm.append(r)
    df = pd.DataFrame(norm, columns=[str(c).strip() for c in cols])
    meta['parser'] = 'csv'
    return df, meta

# ---------- Domain cleaning / mapping ----------
ColMap = Dict[str, str]

def build_cols_map(df: pd.DataFrame) -> ColMap:
    cols_lower = {c.lower(): c for c in df.columns}
    def pick(*names):
        for n in names:
            if n in cols_lower:
                return cols_lower[n]
        return None
    m = {
        'order_id': pick('order-id','order id'),
        'order_item_id': pick('order-item-id','order item id'),
        'product_name': pick('product-name','product name','title'),
        'sku': pick('sku'),
        'asin': pick('asin'),
        'qty': pick('quantity-purchased','qty','quantity'),
        'recipient': pick('recipient-name','recipient','name'),
        'addr1': pick('ship-address-1','address-1'),
        'addr2': pick('ship-address-2','address-2'),
        'addr3': pick('ship-address-3','address-3'),
        'city': pick('ship-city','city'),
        'state': pick('ship-state','state'),
        'postal': pick('ship-postal-code','postal code'),
        'country': pick('ship-country','country'),
        'phone': pick('buyer-phone-number','phone'),
    }
    return m

def clean_str(x: str) -> str:
    return (str(x).replace("\r"," ").replace("\n"," ").strip()) if pd.notna(x) else ""

def product_key(row, m: ColMap) -> str:
    p = clean_str(row.get(m.get('product_name'), ""))
    s = clean_str(row.get(m.get('sku'), ""))
    a = clean_str(row.get(m.get('asin'), ""))
    base = p or s or a or f"Produit {clean_str(row.get(m.get('order_item_id'),''))}"
    return " | ".join([b for b in [p, s, a] if b]) if base else base

# ---------- Text wrapping (width in characters) ----------
def wrap_text(text: str, width: int = 20) -> List[str]:
    if not text:
        return []
    words = text.split()
    lines = []
    cur = ""
    for w in words:
        if len(w) > width:
            if "-" in w:
                parts = w.split("-")
                rebuilt = []
                for i, part in enumerate(parts):
                    if i < len(parts)-1:
                        rebuilt.append(part + "-")
                    else:
                        rebuilt.append(part)
                for segment in rebuilt:
                    if len(segment) > width:
                        i = 0
                        while i < len(segment):
                            chunk = segment[i:i+width]
                            if cur:
                                lines.append(cur)
                                cur = ""
                            lines.append(chunk)
                            i += width
                    else:
                        if not cur:
                            cur = segment
                        elif len(cur) + 1 + len(segment) <= width:
                            cur += " " + segment
                        else:
                            lines.append(cur)
                            cur = segment
            else:
                if cur:
                    lines.append(cur)
                    cur = ""
                i = 0
                while i < len(w):
                    lines.append(w[i:i+width])
                    i += width
        else:
            if not cur:
                cur = w
            elif len(cur) + 1 + len(w) <= width:
                cur += " " + w
            else:
                lines.append(cur)
                cur = w
    if cur:
        lines.append(cur)
    return lines

def format_address(row, m: ColMap) -> List[str]:
    parts = [
        clean_str(row.get(m.get('recipient'), "")),
        clean_str(row.get(m.get('addr1'), "")),
        clean_str(row.get(m.get('addr2'), "")),
        clean_str(row.get(m.get('addr3'), "")),
        " ".join([w for w in [clean_str(row.get(m.get('postal'), "")), clean_str(row.get(m.get('city'), ""))] if w]),
        clean_str(row.get(m.get('state'), "")),
        clean_str(row.get(m.get('country'), "")),
    ]
    parts = [p for p in parts if p]
    wrapped = []
    for p in parts:
        wrapped.extend(wrap_text(p, width=20))
    return [l for l in wrapped if l.strip()]

def unique_orders(df: pd.DataFrame, m: ColMap) -> pd.DataFrame:
    oid_col = m.get('order_id')
    if not oid_col:
        return df.drop_duplicates()
    keep_cols = list({c for c in [
        oid_col, m.get('recipient'), m.get('addr1'), m.get('addr2'), m.get('addr3'),
        m.get('city'), m.get('state'), m.get('postal'), m.get('country')
    ] if c})
    df2 = df.copy()
    if 'purchase-date' in df2.columns:
        try:
            df2 = df2.sort_values('purchase-date')
        except Exception:
            pass
    dfu = df2.drop_duplicates(subset=[oid_col], keep='first')
    return dfu[keep_cols]

# ---------- PDF layout ----------
@dataclass
class PdfLayout:
    margin_left: float = 10 * mm   # tighter left margin so logo sits near edge
    margin_top: float = 15 * mm
    cols: int = 2
    rows: int = 4
    col_gap: float = 8 * mm
    row_gap: float = 6 * mm
    cell_w: float = (A4[0] - 2*10*mm - 8*mm) / 2
    cell_h: float = (A4[1] - 2*15*mm - 6*mm*3) / 4

DEFAULT_LOGO_URL = "https://i.postimg.cc/f3Wwd4hz/default_logo.png"  # direct hosting path; fallback to user's link

def fetch_default_logo_bytes() -> Optional[bytes]:
    try:
        # try to retrieve the image; may fail in restricted envs
        with urllib.request.urlopen(DEFAULT_LOGO_URL, timeout=6) as resp:
            data = resp.read()
            return data
    except Exception:
        # try user's original link (postimg.cc page) as fallback (may not return direct image)
        try:
            with urllib.request.urlopen("https://postimg.cc/f3Wwd4hz", timeout=6) as resp:
                html = resp.read().decode('utf-8', errors='ignore')
                # naive search for an image src in the page
                m = re.search(r'(https?://i\.postimg\.cc/[^\"]+)', html)
                if m:
                    url = m.group(1)
                    with urllib.request.urlopen(url, timeout=6) as r2:
                        return r2.read()
        except Exception:
            pass
    return None

def generate_labels_pdf(df_orders: pd.DataFrame, m: ColMap, out_path="etiquettes.pdf", logo_bytes: Optional[bytes]=None) -> Tuple[str, int]:
    c = canvas.Canvas(out_path, pagesize=A4)
    layout = PdfLayout()
    logo = None
    # if no provided logo, try to fetch default once
    if not logo_bytes:
        fetched = fetch_default_logo_bytes()
        if fetched:
            logo_bytes = fetched
    if logo_bytes:
        try:
            logo = ImageReader(io.BytesIO(logo_bytes))
        except Exception:
            logo = None

    n = 0
    # adjust logo factors: 20% larger than previous setting
    base_max_h_factor = 0.6
    base_max_w_factor = 0.28
    max_h_factor = base_max_h_factor * 1.2
    max_w_factor = base_max_w_factor * 1.2
    for idx, row in df_orders.iterrows():
        page_idx = n // (layout.cols * layout.rows)
        pos_in_page = n % (layout.cols * layout.rows)
        col = pos_in_page % layout.cols
        rw = pos_in_page // layout.cols
        x = layout.margin_left + col * (layout.cell_w + layout.col_gap)
        y = A4[1] - layout.margin_top - (rw+1)*layout.cell_h - rw*layout.row_gap

        # Logo on left if present (glued to left, small left inset)
        logo_width = 0
        logo_height = 0
        if logo:
            try:
                max_h = layout.cell_h * max_h_factor
                max_w = layout.cell_w * max_w_factor
                try:
                    pil = Image.open(io.BytesIO(logo_bytes))
                    iw, ih = pil.size
                    ratio = iw / float(ih) if ih != 0 else 1.0
                except Exception:
                    ratio = 1.0
                if ratio >= 1:
                    logo_width = min(max_w, max_h * ratio)
                    logo_height = logo_width / ratio
                else:
                    logo_height = min(max_h, max_w / ratio)
                    logo_width = logo_height * ratio
                # draw image very close to left border of cell
                lx = x + 2  # glued left
                ly = y + (layout.cell_h - logo_height)/2  # centered vertically
                c.drawImage(logo, lx, ly, width=logo_width, height=logo_height, preserveAspectRatio=True, mask='auto')
            except Exception:
                logo_width = 0
                logo_height = 0

        # Address — bold and larger, centered vertically inside cell
        addr_lines = format_address(row, m)
        font_name_b = "Helvetica-Bold"
        font_name = "Helvetica"
        font_size = 13  # slightly larger per request
        leading = font_size + 2

        n_lines = len(addr_lines) if addr_lines else 1
        text_block_h = n_lines * leading
        text_x = x + 6 + (logo_width + 6 if logo_width else 0)  # gap after logo
        # center vertically: compute top baseline
        text_y_top = y + (layout.cell_h + text_block_h)/2
        text_obj = c.beginText()
        text_obj.setTextOrigin(text_x, text_y_top - font_size)
        text_obj.setLeading(leading)
        for i, ln in enumerate(addr_lines):
            if i == 0:
                text_obj.setFont(font_name_b, font_size+1)
                text_obj.textLine(ln)
                text_obj.setFont(font_name, font_size)
            else:
                text_obj.textLine(ln)
        c.drawText(text_obj)

        n += 1
        if n % (layout.cols*layout.rows) == 0:
            c.showPage()

    if n % (layout.cols*layout.rows) != 0:
        c.showPage()
    c.save()
    return out_path, n

# ---------- Grouping and checkboxes ----------
def build_group_key(row, m: ColMap) -> str:
    parts = []
    p = clean_str(row.get(m.get('product_name'), "")) or ""
    s = clean_str(row.get(m.get('sku'), "")) or ""
    a = clean_str(row.get(m.get('asin'), "")) or ""
    if p: parts.append(p)
    if s: parts.append(s)
    if a: parts.append(a)
    if not parts:
        parts = [f"Produit {clean_str(row.get(m.get('order_item_id'),''))}"]
    return " | ".join(parts)

def summarize_groups(df: pd.DataFrame, m: ColMap) -> pd.DataFrame:
    if df.empty:
        return pd.DataFrame(columns=['_group','count'])
    tmp = df.copy()
    tmp['_group'] = tmp.apply(lambda r: build_group_key(r, m), axis=1)
    oid = m.get('order_id')
    if oid and oid in tmp.columns:
        grp = tmp.groupby('_group')[oid].nunique().reset_index().rename(columns={oid:'count'})
    else:
        grp = tmp.groupby('_group').size().reset_index(name='count')
    grp = grp.sort_values(['count','_group'], ascending=[False, True])
    return grp

# ---------- Global state ----------
df_raw_global: Optional[pd.DataFrame] = None
cols_map_global: Optional[ColMap] = None
logo_bytes_global: Optional[bytes] = None
checkboxes: List[Checkbox] = []
groups_box = VBox()
groups_bar = HBox(layout=Layout(gap="8px"))
stats_html = HTML()
export_out = Output()

_FLAG_MAP = {
    'FR': '🇫🇷', 'FRANCE': '🇫🇷',
    'GB': '🇬🇧', 'UK': '🇬🇧', 'UNITED KINGDOM': '🇬🇧',
    'US': '🇺🇸', 'UNITED STATES': '🇺🇸', 'USA': '🇺🇸',
    'DE': '🇩🇪', 'GERMANY': '🇩🇪',
    'ES': '🇪🇸', 'SPAIN': '🇪🇸',
    'IT': '🇮🇹', 'ITALY': '🇮🇹',
}

def country_flag_label(country_name: str) -> str:
    if not country_name:
        return '🏳️'
    k = country_name.strip().upper()
    if k in _FLAG_MAP:
        return _FLAG_MAP[k]
    if len(k) == 2 and k.isalpha():
        try:
            return ''.join(chr(0x1F1E6 + ord(ch)-65) for ch in k)
        except Exception:
            return '🏳️'
    for kk in _FLAG_MAP:
        if kk in k:
            return _FLAG_MAP[kk]
    return '🏳️'

def render_stats(df_raw: pd.DataFrame, df_orders_unique: pd.DataFrame, m: ColMap, groups_df):
    dfu = df_orders_unique if df_orders_unique is not None and not df_orders_unique.empty else unique_orders(df_raw, m) if df_raw is not None else pd.DataFrame()
    country_col = m.get('country')
    country_counts = {}
    if country_col and country_col in dfu.columns:
        country_counts = dfu[country_col].astype(str).str.strip().replace('', '—').value_counts().to_dict()
    total_cmd = sum(country_counts.values()) if country_counts else (dfu.shape[0] if not dfu.empty else 0)
    rows_html = ""
    if country_counts:
        for k, v in sorted(country_counts.items(), key=lambda x: -x[1]):
            flag = country_flag_label(k)
            rows_html += f"<div style='display:flex;align-items:center;margin:4px 0'><div style='font-size:18px;margin-right:8px'>{flag}</div><div style='font-size:14px;color:#213547'>{k}</div><div style='margin-left:auto;font-weight:700;color:#0b3954'>{v}</div></div>"
    else:
        rows_html = "<div style='color:#7a8696'>Aucune commande détectée.</div>"
    stats_html.value = f"""
    <div style="font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, 'Helvetica Neue', Arial; background:#fbfbfd;border:1px solid #e8eef6;border-radius:10px;padding:12px;">
      <div style="display:flex;align-items:baseline;gap:10px">
        <div style="font-size:13px;color:#6b7785">Commandes uniques</div>
        <div style="font-size:20px;font-weight:700;color:#0b3954">{total_cmd}</div>
      </div>
      <div style="margin-top:8px">{rows_html}</div>
    </div>
    """

def refresh_groups_ui(df: pd.DataFrame, m: ColMap):
    global checkboxes
    checkboxes = []
    grp = summarize_groups(df, m)
    if grp.empty:
        groups_box.children = [HTML("<i>Aucun produit détecté.</i>")]
        return
    items = []
    for _, r in grp.iterrows():
        label = f"[{int(r['count'])}] {r['_group']}"
        cb = Checkbox(value=True, description=label, indent=False, layout=Layout(width="100%"))
        items.append(cb)
        checkboxes.append(cb)
    groups_box.children = items

def apply_selection(df: pd.DataFrame, m: ColMap) -> pd.DataFrame:
    if df is None:
        return pd.DataFrame()
    if not checkboxes:
        return unique_orders(df, m)
    enabled = set()
    for cb in checkboxes:
        if cb.value:
            label = cb.description
            gname = re.sub(r'^\[\d+\]\s*', '', label)
            enabled.add(gname)
    tmp = df.copy()
    tmp['_group'] = tmp.apply(lambda r: build_group_key(r, m), axis=1)
    tmp = tmp[tmp['_group'].isin(enabled)]
    df_orders = unique_orders(tmp, m)
    return df_orders

# ---------- Widgets factory (allow rebuild on RESET) ----------
def make_widgets():
    title = HTML("""<div style="font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, 'Helvetica Neue', Arial;">
        <div style="font-size:22px;font-weight:700;color:#0b3954">Osmoz <span style="color:#1273de">1.22</span></div>
        <div style="color:#7a8696">Import Amazon + Étiquettes PDF</div>
    </div>""")

    hint = HTML("""<div style="background:#f6f9fe;border:1px solid #eef5ff;border-radius:8px;padding:8px 10px;font-size:12.5px;color:#1f3b57">
        1) Déposez l’export Amazon (.txt/.csv/.tsv/.xlsx). 2) Cliquez <b>Charger les données</b>.<br>
        3) Cochez les produits (groupés). 4) Cliquez <b>Générer le PDF</b>.
    </div>""")

    uploader = FileUpload(
        accept=".txt,.csv,.tsv,.xlsx",
        multiple=False,
        description="Fichier Amazon",
        layout=Layout(width="78%", height="48px"),
    )

    uploader_logo = FileUpload(
        accept=".png,.jpg,.jpeg",
        multiple=False,
        description="Logo (option)",
        layout=Layout(width="38%", height="48px"),
    )

    btn_load = Button(
        description="📥 Charger les données",
        button_style="",
        layout=Layout(width="50%", height="42px"),
        style={'button_color': '#daf7e6'}
    )
    btn_pdf = Button(
        description="📄 Générer le PDF",
        button_style="",
        layout=Layout(width="48%", height="42px"),
        style={'button_color': '#eaf4ff'}
    )
    btn_reset = Button(
        description="RESET",
        button_style="danger",
        layout=Layout(width="48%", height="36px")
    )
    return title, hint, uploader, uploader_logo, btn_load, btn_pdf, btn_reset

# ---------- Upload helpers ----------
def get_first_upload(fu: FileUpload) -> Optional[Tuple[bytes, str]]:
    v = fu.value
    if not v:
        return None
    if isinstance(v, (tuple, list)):
        up = v[0]
        content = getattr(up, "content", None)
        name = getattr(up, "name", None)
        if content is None and isinstance(up, dict):
            content = up.get('content')
            name = up.get('name') or up.get('metadata', {}).get('name')
        return (to_bytes(content), name or "upload.bin")
    if isinstance(v, dict):
        up = next(iter(v.values()))
        content = up.get('content')
        name = up.get('metadata', {}).get('name') or up.get('name') or "upload.bin"
        return (to_bytes(content), name)
    return None

def load_logo_from_uploader(uploader_logo: FileUpload) -> Optional[bytes]:
    up = get_first_upload(uploader_logo)
    if not up:
        return None
    content, _ = up
    return to_bytes(content) if content is not None else None

# ---------- Actions and UI build ----------
def build_interface():
    global df_raw_global, cols_map_global, logo_bytes_global
    global groups_box, groups_bar, stats_html, export_out

    df_raw_global = None
    cols_map_global = None
    logo_bytes_global = None

    title, hint, uploader, uploader_logo, btn_load, btn_pdf, btn_reset = make_widgets()
    stats_html = HTML()
    groups_box = VBox()
    groups_bar = HBox(layout=Layout(gap="8px"))
    export_out = Output()

    groups_bar.children = [
        btn_pdf,
        btn_reset,
    ]

    def action_load_clicked(_):
        nonlocal uploader, uploader_logo
        try:
            up = get_first_upload(uploader)
            if not up:
                with log:
                    print("Aucun fichier importé.")
                return
            (file_bytes, filename) = up
            with log:
                print(f"Chargement: {filename}")
            df, meta = read_table_flex(file_bytes, filename)
            m = build_cols_map(df)
            oid = m.get('order_id')
            if oid and oid in df.columns:
                try:
                    df = df[df[oid].astype(str).str.match(r'^\d{3}-\d{7}-\d{7}$', na=False)]
                except Exception:
                    pass
            for k in ['recipient','addr1','addr2','addr3','city','state','postal','country','product_name','sku']:
                col = m.get(k)
                if col and col in df.columns:
                    df[col] = df[col].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
            logo = load_logo_from_uploader(uploader_logo)
            global df_raw_global, cols_map_global, logo_bytes_global
            df_raw_global = df.reset_index(drop=True)
            cols_map_global = m
            logo_bytes_global = logo
            refresh_groups_ui(df_raw_global, cols_map_global)
            df_sel = apply_selection(df_raw_global, cols_map_global)
            render_stats(df_raw_global, df_sel, cols_map_global, None)
            with log:
                print("Import terminé. Groupes produits générés.")
        except Exception as e:
            with log:
                print("Erreur chargement:", e)
                traceback.print_exc()

    def action_pdf_clicked(_):
        try:
            if df_raw_global is None or cols_map_global is None:
                with log: print("Importez d’abord un fichier, puis cliquez sur Charger les données.")
                return
            df_sel_orders = apply_selection(df_raw_global, cols_map_global)
            if df_sel_orders.empty:
                with log: print("Aucun résultat: vérifiez les cases cochées.")
                return
            out_path, n = generate_labels_pdf(df_sel_orders, cols_map_global, out_path="etiquettes.pdf", logo_bytes=logo_bytes_global)
            with export_out:
                clear_output(wait=True)
                # Display file link prominently (no download button)
                try:
                    fl = FileLink(out_path)
                    display(HTML(f"<div style='margin:6px 0 8px 0; font-size:15px; display:flex; align-items:center; gap:10px;'>📥 <b style=\"font-size:15px;color:#0b3954\">etiquettes.pdf</b> — <span style='font-weight:700;color:#0b3954'>{n} étiquette(s)</span></div>"))
                    display(fl)
                except Exception:
                    display(HTML(f"<div style='margin:6px 0 8px 0; font-size:15px;'>📥 <b>etiquettes.pdf</b> — {n} étiquette(s)</div>"))
            with log:
                print("PDF généré avec succès.")
            render_stats(df_raw_global, df_sel_orders, cols_map_global, None)
        except Exception as e:
            with log:
                print("Erreur génération PDF:", e)
                traceback.print_exc()

    def action_reset_clicked(_):
        global df_raw_global, cols_map_global, logo_bytes_global
        df_raw_global = None
        cols_map_global = None
        logo_bytes_global = None
        with export_out:
            clear_output(wait=True)
        with log:
            clear_output(wait=True)
            print("Interface réinitialisée.")
        build_interface()

    btn_load.on_click(action_load_clicked)
    btn_pdf.on_click(action_pdf_clicked)
    btn_reset.on_click(action_reset_clicked)

    left_col = VBox([
        HTML("<h4 style='margin:10px 0 6px 0;'>Import</h4>"),
        HBox([uploader, uploader_logo], layout=Layout(gap="8px")),
        HBox([btn_load], layout=Layout(margin="8px 0", gap="8px")),
        HTML("<h4 style='margin:10px 0 6px 0;'>Statistiques</h4>"),
        stats_html,
        HTML("<h4 style='margin:12px 0 6px 0;'>Résultat</h4>"),
        export_out,
    ], layout=Layout(width="46%"))

    right_col = VBox([
        HTML("<h4 style='margin:10px 0 6px 0;'>Sélection des produits</h4>"),
        groups_bar,
        groups_box,
        HTML("<h4 style='margin:12px 0 6px 0;'>Journal</h4>"),
        log
    ], layout=Layout(width="52%"))

    app_box.children = [
        title,
        hint,
        HBox([left_col, right_col], layout=Layout(justify_content="space-between", gap="16px"))
    ]

# build and display UI
build_interface()
display(app_box)
with log:
    print("Interface prête. 1) Déposez l’export. 2) Cliquez “Charger les données”. 3) Cochez les produits. 4) Cliquez “Générer le PDF”.")


Installation de pillow…
